![imagenes](logo.png)

# Codificación de variables categóricas en Machine Learning

En un conjunto de datos no todas las columnas son numéricas. Es común encontrar variables como **color**, **ciudad**, **tipo de cliente** o **género**, que son **categóricas**. El problema es que la mayoría de los algoritmos de Machine Learning no saben trabajar con texto: requieren entradas numéricas.  

Una tentación podría ser reemplazar las categorías con números arbitrarios (ejemplo: rojo = 1, azul = 2, verde = 3). Pero esto introduce un error: el modelo interpretará que **existe un orden o una distancia entre las categorías** que en realidad no tiene sentido (¿acaso “azul” es mayor que “rojo”?).  

Por ello, necesitamos **métodos de codificación** que traduzcan las categorías a números de una manera adecuada para el algoritmo, sin inventar relaciones inexistentes.

Piensa en una reunión con personas que hablan distintos idiomas: español, inglés y francés. Para colaborar, necesitan un traductor que unifique todo en un mismo idioma.  
La codificación es ese traductor: convierte categorías en un “idioma numérico” comprensible para el algoritmo.

Un teatro tiene localidades: **Orquesta**, **Platea**, **Balcón**. Si las numeramos como 1, 2 y 3, parecería que hay una jerarquía: Balcón > Platea > Orquesta.  
En realidad solo son etiquetas. La codificación correcta sería dar a cada sección su propio boleto sin relación de magnitud entre ellas.

---

## Métodos principales de codificación

Existen varias formas de transformar categorías en números. Los más usados en la práctica son:

### 1. One-Hot Encoding (codificación en dummies)

Se crea una columna binaria por cada categoría.  
Si `Color` puede ser *Rojo, Azul, Verde*, el resultado es:

| Color_Rojo | Color_Azul | Color_Verde |
|------------|------------|-------------|
| 1          | 0          | 0           |
| 0          | 1          | 0           |
| 0          | 0          | 1           |

Cada fila tiene un 1 en la columna correspondiente y 0 en las demás.  Este proceso no introduce órdenes ficticios.  Funciona muy bien para variables nominales (sin orden).  Sin embargo, aumenta mucho el número de columnas si hay muchas categorías.  

---

### 2. Ordinal Encoding (codificación ordinal)

Cada categoría se asigna a un número entero, respetando un orden natural. Ejemplo:

- *Bajo* → 1  
- *Medio* → 2  
- *Alto* → 3  

Entre las ventajas están que solo ocupa una sola columna y es útil cuando sí hay un orden intrínseco.  Pero si no existe orden real, puede engañar al modelo.  

---

## ¿Cuándo usar cada una?

- **Nominal (sin orden):** One-Hot Encoding. Ej.: color, ciudad, género.  
- **Ordinal (con orden real):** Ordinal Encoding. Ej.: talla de ropa, nivel educativo.  

---

## El problema de la fuga de datos

Al igual que con el escalado, aquí también puede haber **fuga de datos**.  
Si ajustamos un codificador usando todo el dataset, el conjunto de prueba deja de ser “desconocido”. Por ejemplo, con Target Encoding, el promedio de la variable objetivo en el conjunto completo ya filtra información del test hacia el entrenamiento.  

La práctica correcta es siempre:  
1. Separar en entrenamiento y prueba.  
2. Ajustar (`fit`) el codificador solo con el entrenamiento.  
3. Aplicar (`transform`) esa misma transformación a entrenamiento y prueba.  

## Codificación en Python

De https://github.com/scidatmath2020/ML_Py_25/tree/main/data descarga la tabla ``datos_para_codificar.csv``. Con ella, veremos cómo se hace el escalado de columnas numéricas en Python.

In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

datos = pd.read_csv("datos_para_codificar_ALUMNOS.csv")
datos

,producto,sede,turno,nivel_prioridad
0,A,Norte,Mañana,Baja
1,B,Centro,Tarde,Media
2,C,Sur,Noche,Alta
3,A,Centro,Noche,Media
4,B,Norte,Mañana,Alta
5,C,Centro,Tarde,Baja
6,B,Sur,Mañana,Media
7,A,Sur,Tarde,Alta


In [8]:
# Columnas
cols_onehot  = ["producto", "sede","turno"]                               # NOMINALES → One-Hot
cols_ordinal = ["nivel_prioridad"]      # ORDINAS → Ordinal

# Categorías ordenadas para las ordinales (mismo orden que en cols_ordinal)
categorias_ordinales = [
    ["Baja", "Media", "Alta"]
]

'''
Explicación del ColumnTransformer usado

1) transformers = [ ... ]
   Cada tupla es (nombre_bloque, transformador, columnas_objetivo)

   a) ("onehot", OneHotEncoder(...), cols_onehot)
      - Aplica One-Hot Encoding a columnas nominales.
      - sparse_output=False  → salida densa (más cómodo para DataFrame).
      - drop=None            → no se descarta ninguna categoría.
      - handle_unknown="ignore" → si aparece una categoría NO vista en fit,
        no lanza error; la fila queda con ceros en todas las dummies de esa variable.

   b) ("ordinal", OrdinalEncoder(...), cols_ordinal)
      - Aplica codificación ordinal respetando un ORDEN explícito por columna.
      - categories=categorias_ordinales → listas con el orden de cada variable.
      - handle_unknown="use_encoded_value", unknown_value=-1  → cualquier
        categoría NO vista en fit se codifica como -1 (fuera del rango normal 0..k-1).
        Esto permite detectar fácilmente valores "desconocidos" en datos nuevos.

Resumen:
- One-Hot para nominales (sin orden), tolerante a categorías nuevas (se codifican como todo ceros).
- Ordinal para variables con orden natural; categorías nuevas se marcan con -1.
- El resto de columnas se descartan (remainder="drop").
'''

preprocesador = ColumnTransformer(
    transformers=[
        ("onehot",
         OneHotEncoder(sparse_output=False, drop=None, handle_unknown="ignore"),
         cols_onehot),
        ("ordinal",
         OrdinalEncoder(categories=categorias_ordinales,
                        handle_unknown="use_encoded_value", unknown_value=-1),
         cols_ordinal),
    ],
    remainder="drop",
    verbose_feature_names_out=False
)

preprocesador.fit(datos)

X_encoded = preprocesador.transform(datos)

In [9]:
X_encoded

array([[1., 0., 0., 0., 1., 0., 1., 0., 0., 0.],
       [0., 1., 0., 1., 0., 0., 0., 0., 1., 1.],
       [0., 0., 1., 0., 0., 1., 0., 1., 0., 2.],
       [1., 0., 0., 1., 0., 0., 0., 1., 0., 1.],
       [0., 1., 0., 0., 1., 0., 1., 0., 0., 2.],
       [0., 0., 1., 1., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 1., 1., 0., 0., 1.],
       [1., 0., 0., 0., 0., 1., 0., 0., 1., 2.]])

In [10]:
# Reconstruir DataFrame con nombres de columnas
cols_out = preprocesador.get_feature_names_out()
df_encoded = pd.DataFrame(X_encoded, columns=cols_out, index=datos.index)
df_encoded

,producto_A,producto_B,producto_C,sede_Centro,sede_Norte,sede_Sur,turno_Mañana,turno_Noche,turno_Tarde,nivel_prioridad
0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
4,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0
5,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
6,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
7,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0


In [11]:
df_encoded.to_csv("dat_cat.csv",index=False)